In [10]:
import os
os.chdir(r"c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination_sandbox")
import re

import gensim
import numpy as np
import pandas as pd
import pickle
from nltk.corpus import wordnet, stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from sklearn.feature_extraction.text import TfidfVectorizer
from unidecode import unidecode

In [9]:
google_model = gensim.models.KeyedVectors.load_word2vec_format("c:/Users/britt/Downloads/GoogleNews-vectors-negative300.bin.gz", binary=True)

In [11]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith("J"):
        return wordnet.ADJ
    elif treebank_tag.startswith("V"):
        return wordnet.VERB
    elif treebank_tag.startswith("N"):
        return wordnet.NOUN
    elif treebank_tag.startswith("R"):
        return wordnet.ADV
    else:
        return ""

def penn_to_wn(tag):
    return get_wordnet_pos(tag)

def preprocess_document(doc):
    lemmatizer = WordNetLemmatizer()

    remove_https = re.sub(r"http\S+", "", doc)
    remove_com = re.sub(r"\ [A-Za-a]*\.com", " ", remove_https)
    remove_numbers_punctuations = re.sub(r"[^a-zA-Z]+", " ", remove_com)
    pattern = re.compile(r"\s+")
    remove_extra_whitespaces = re.sub(pattern, " ", remove_numbers_punctuations)
    only_ascii = unidecode(remove_extra_whitespaces)
    doc = only_ascii.lower()

    list_of_tokens = word_tokenize(doc)
    list_of_tokens_pos = pos_tag(list_of_tokens)
    list_of_tokens_wn_pos = [(token[0], penn_to_wn(token[1])) for token in list_of_tokens_pos if token[0] not in stopwords.words("english")]
    list_of_lemmas = [lemmatizer.lemmatize(token[0], token[1]) if token[1] != "" else lemmatizer.lemmatize(token[0]) for token in list_of_tokens_wn_pos]
    cleaned_text = [" ".join(list_of_lemmas)]

    return cleaned_text

In [12]:
df = pd.read_csv("data/cleaned_data/bulletins_labels_share_content.csv", dtype={'ID': object})
corpus = list(df["Cleaned text"])
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit_transform(corpus)

<177x3835 sparse matrix of type '<class 'numpy.float64'>'
	with 68003 stored elements in Compressed Sparse Row format>

In [14]:
file_name = "ASBESTOS WORKER 3435 100518.txt"

with open(f"data/original_data/job_bulletins/{file_name}", "r", encoding="utf-8") as f:
    application = f.read()

text = preprocess_document(application)
regressor_vocabulary = tfidf_vectorizer.get_feature_names_out()

scaled_embeddings  = []
doc_list = text[0].split()
for word in doc_list:
    if word in google_model.key_to_index.keys():
        embedding = google_model[word]
        index = np.where(regressor_vocabulary == word)[0]
        try:
            scaled_embeddings.append(embedding * tfidf_vectorizer.idf_[index])
        except ValueError:
            pass

doc_vector = np.average(scaled_embeddings, axis=0)

In [18]:
with open("data/models/cat_boost_regr.pkl", "rb") as read_file:
    regressor = pickle.load(read_file)

In [19]:
doc_vector.shape

(300,)

In [21]:
regressor.predict(doc_vector)

array([0.82054963, 0.17006495])